In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

file_path = r"C:\Users\aamir\Desktop\YC\P\ChurnPrediction\src\infrastructure\data\dataset-68f599a4c9b84581895311-6907667a3f0be705109390.csv"

df = spark.read.csv(file_path,header=True,inferSchema=True)
df.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|

In [5]:
dublucatedata =  df.groupBy(df.columns).count()

dublucatedata.show()

+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+-----+
|RowNumber|CustomerId|   Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|count|
+---------+----------+----------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+-----+
|       79|  15575185|   Bushell|        757|    Spain|  Male| 33|     5| 77253.22|            1|        0|             1|      194239.63|     0|    1|
|      396|  15807432|     Cheng|        645|  Germany|Female| 37|     2|136925.09|            2|        0|             1|      153400.24|     0|    1|
|      503|  15714485|   Udinese|        774|   France|  Male| 60|     5| 85891.55|            1|        1|             0|       74135.48|     1|    1|
|      746|  15640059|     Smith|        606|   France|  Male| 40|     5|      0.0|     

In [6]:
supp_column = ["RowNumber","CustomerId","Surname"]

df_supp = df.drop(*supp_column)

print("data is deleted succ ")
df_supp.show()





data is deleted succ 
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        850|    Spain|Female| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
|        645|    Spain|  Male| 44|     8|113755.78|            2| 

In [7]:
df.filter(df.Balance==0).count()


3617

In [8]:
	
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

indexer_gender = StringIndexer(inputCol="gender", outputCol="gender_num")
df_indexed = indexer_gender.fit(df).transform(df)

indexer_geo = StringIndexer(inputCol="Geography", outputCol="GeographyIndex")
df_indexed = indexer_geo.fit(df_indexed).transform(df_indexed) 

df_final = df_indexed.withColumn("Geography_France", (col("GeographyIndex") == 0).cast("int")) \
.withColumn("Geography_Germany", (col("GeographyIndex") == 1).cast("int")) \
.withColumn("Geography_Spain", (col("GeographyIndex") == 2).cast("int")) \
.drop("gender", "Geography", "GeographyIndex") \
.withColumnRenamed("gender_num", "gender")

df_final.show()
df_final.printSchema()

+---------+----------+---------+-----------+---+------+---------+-------------+---------+--------------+---------------+------+------+----------------+-----------------+---------------+
|RowNumber|CustomerId|  Surname|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|gender|Geography_France|Geography_Germany|Geography_Spain|
+---------+----------+---------+-----------+---+------+---------+-------------+---------+--------------+---------------+------+------+----------------+-----------------+---------------+
|        1|  15634602| Hargrave|        619| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|   1.0|               1|                0|              0|
|        2|  15647311|     Hill|        608| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|   1.0|               0|                0|              1|
|        3|  15619304|     Onio|        502| 42|     8| 159660.8|     

In [10]:
df_final.toPandas().to_csv("../src/infrastructure/data/clean_data.csv", index=False)